In [ ]:
# legacy_path = "stack_cubes/images"
# import os
# for filen in os.listdir(legacy_path):
#     if filen.endswith(".jpg") and "_" in filen:
#         os.remove(os.path.join(legacy_path, filen))

In [2]:
from internvl_eval.utils import ManiSkillTrajectoryDataset, InternVLPretrainDatasetGenerator

# joint_dataset = ManiSkillTrajectoryDataset(dataset_file="demos/StackCube-v1/motionplanning/trajectory.rgb.pd_joint_delta_pos.physx_cpu.h5", success_only=False, device=None, load_count=100, is_episode_dataset=True)
# eef_dataset = ManiSkillTrajectoryDataset(dataset_file="/mnt/nfs3/caozhe/workspace/ManiSkill/demos/StackCube-v1/motionplanning/trajectory.rgb.pd_ee_delta_pose.physx_cpu.h5", success_only=False, device=None, load_count=100, is_episode_dataset=True)
push_dataset = ManiSkillTrajectoryDataset(dataset_file="demos/PushCube-v1/motionplanning/trajectory.rgb.pd_ee_delta_pose.physx_cpu.h5", success_only=False, device=None, load_count=500, is_episode_dataset=True)

100%|██████████| 500/500 [02:13<00:00,  3.76it/s]


In [3]:
generator = InternVLPretrainDatasetGenerator(
    dataset=push_dataset,
    save_path="push_cubes500",
    horizon=1,
    dual_camera=False,
    is_joint_action=False,
    env_id="PushCube-v1"
    )
# generator.cal_statistics()
generator.traj_generation(filter_zero=True)
# generator.generation()

In [4]:
joint_dataset = ManiSkillTrajectoryDataset(dataset_file="demos/StackCube-v1/motionplanning/trajectory.rgb.pd_ee_delta_pose.physx_cpu.h5", success_only=False, device=None, load_count=100, is_episode_dataset=True)

100%|██████████| 100/100 [01:56<00:00,  1.16s/it]


In [ ]:
generator = InternVLPretrainDatasetGenerator(
    dataset=joint_dataset,
    save_path="stack_cubes_100",
    horizon=1,
    dual_camera=False,
    is_joint_action=False,
    env_id="StackCube-v1"
    )
# generator.cal_statistics()
generator.traj_generation(filter_zero=True)
# generator.generation()

In [33]:
import copy

from ray import get
def get_seq_dataset(dataset):
    new_dataset = copy.copy(dataset)
    sequence_obs = None
    for obs in new_dataset.obs:
        if sequence_obs is None:
            sequence_obs = obs
        else:
            sequence_obs = common.append_dict_array(sequence_obs, obs)
    new_dataset.obs = sequence_obs
    new_dataset.actions = np.vstack(new_dataset.actions)
    new_dataset.terminated = np.concatenate(new_dataset.terminated)
    new_dataset.truncated = np.concatenate(new_dataset.truncated)
    if new_dataset.rewards is not None:
        new_dataset.rewards = np.concatenate(new_dataset.rewards)
    if new_dataset.success is not None:
        new_dataset.success = np.concatenate(new_dataset.success)
    if new_dataset.fail is not None:
        new_dataset.fail = np.concatenate(new_dataset.fail)
    new_dataset.is_episode_dataset = False
    return new_dataset
eef_dataset = dataset
eef_sq_ds = get_seq_dataset(eef_dataset)

In [ ]:
generator = InternVLPretrainDatasetGenerator(
    dataset=eef_sq_ds,
    save_path="push_cubes",
    horizon=1,
    dual_camera=False,
    is_joint_action=False,
    )
# generator.cal_statistics()
generator.traj_generation()
# generator.generation()

In [6]:
import mani_skill.envs
import gymnasium as gym
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Set to the GPU you want to use

N = 256
env = gym.make("PickCube-v1", num_envs=N, reconfiguration_freq=None)
env = ManiSkillVectorEnv(env, num_envs=N, ignore_terminations=False, auto_reset=True, record_metrics=True)
env.action_space # shape (N, D)
env.single_action_space # shape (D, )
env.observation_space # shape (N, ...)
env.single_observation_space # shape (...)
env.reset()
obs, rew, terminated, truncated, info = env.step(env.action_space.sample())
# obs (N, ...), rew (N, ), terminated (N, ), truncated (N, )

In [5]:
import mani_skill.envs
import gymnasium as gym
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv
N = 4
env = gym.make("PickCube-v1", num_envs=N, max_episode_steps=50)
env = ManiSkillVectorEnv(env, auto_reset=True, ignore_terminations=False)
env.action_space # shape (N, D)
env.single_action_space # shape (D, )
env.observation_space # shape (N, ...)
env.single_observation_space # shape (...)
env.reset()
for i in range(100):
    obs, rew, terminated, truncated, info = env.step(env.action_space.sample())
    print(terminated, truncated)
# obs (N, ...), rew (N, ), terminated (N, ), truncated (N, )

tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False, False], device='cuda:0') tensor([False, False, False, False], device='cuda:0')
tensor([False, False, False,

In [ ]:
import gymnasium as gym
import torch
import mani_skill.envs
from collections import defaultdict
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv
env_id = "PushCube-v1"
num_eval_envs = 64
env_kwargs = dict(obs_mode="rgb") # modify your env_kwargs here
eval_envs = gym.make(env_id, num_envs=num_eval_envs, reconfiguration_freq=None, **env_kwargs)
# add any other wrappers here
eval_envs = ManiSkillVectorEnv(eval_envs, ignore_terminations=False, record_metrics=True)

# evaluation loop, which will record metrics for complete episodes only
obs, _ = eval_envs.reset(seed=0)
eval_metrics = defaultdict(list)
for _ in range(450):
    action = eval_envs.action_space.sample() # replace with your policy action
    obs, rew, terminated, truncated, info = eval_envs.step(action)
    # note as there are no partial resets, truncated is True for all environments at the same time
    print("step")
    if truncated.any() or terminated.any():
        for k, v in info["final_info"]["episode"].items():
            eval_metrics[k].append(v.float())
for k in eval_metrics.keys():
    print(f"{k}_mean: {torch.mean(torch.stack(eval_metrics[k])).item()}")

NameNotFound: Environment `PushCube` doesn't exist. Did you mean: `Pusher`?

In [4]:
for k, v in eval_metrics.items():
    print(f"{k}_mean: {torch.mean(torch.stack(v)).item()}")
    eval_metrics[k] = torch.stack(v).cpu().numpy().tolist()

success_once_mean: 0.0
return_mean: 7.305637359619141
episode_len_mean: 150.0
reward_mean: 0.048704247921705246
success_at_end_mean: 0.0


In [5]:
eval_metrics[k] 

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [ ]:
import gymnasium as gym
import mani_skill.envs
from mani_skill.utils.wrappers.gymnasium import CPUGymWrapper
from PIL import Image
import numpy as np
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv

num_envs = 2
env = gym.make(
    "StackCube-v1", # there are more tasks e.g. "PushCube-v1", "PegInsertionSide-v1", ...
    num_envs=num_envs,
    obs_mode="rgb", # there is also "state_dict", "rgbd", ...
    control_mode="pd_ee_delta_pose", # there is also "pd_joint_delta_pos", ...
    sensor_configs={'height': 480, 'width': 480}, # camera configs
    max_episode_steps=150,
)
env = ManiSkillVectorEnv(env, auto_reset=True, ignore_terminations=False)
# print("Observation space", env.observation_space)
# print("Action space", env.action_space)
# env = CPUGymWrapper(env)  # wrap to use CPU for rendering
obs, _ = env.reset(seed=0) # reset with a seed for determinism
last_obs = obs
num_dones = 16
from internvl_eval.InternVL_eval_agent import VideoRecorder
recorder = VideoRecorder(
    save_path="./test_save",
    fps=30,
    num_envs=num_envs,
)
while num_dones > 0:
    actions = env.action_space.sample()  # sample random actions
    obs, reward, terminated, truncated, info = env.step(actions)  # step with
    is_terminals = np.logical_or(terminated.cpu().numpy(), truncated.cpu().numpy())
    recorder.append_obs(last_obs, info['success'].cpu().numpy(), is_terminals, actions)
    last_obs = obs
    done_env_ids = np.where(is_terminals)[0]
    if len(done_env_ids) > 0:
        obs, _ = env.reset(options={"env_idx": done_env_ids, "reconfigure": True})  # reset the environments that are done
        last_obs = obs
    for env_id in done_env_ids:
        print(f"Environment {env_id} done with reward {reward[env_id]}, num dones: {num_dones}")
        num_dones -= 1
# done = False
# Image.fromarray(obs["sensor_data"]['base_camera']['rgb']).show()  # show the first environment's RGB observation
# while not done:
#     action = env.action_space.sample()
#     obs, reward, terminated, truncated, info = env.step(action)
#     done = terminated or truncated
#     # env.render()  # a display is required to render
# env.close()


# python -m mani_skill.trajectory.replay_trajectory --traj_path demos/StackCube-v1/motionplanning/trajectory.h5 --obs_mode rgb --target_control_mode pd_ee_delta_pose --verbose --save_traj --max_retry 2 --reward_mode 'sparse' --record_rewards --num_envs 32 --count 100
# python -m mani_skill.trajectory.replay_trajectory --traj_path demos/PickCube-v1/motionplanning/trajectory.h5 --obs_mode rgb --target_control_mode pd_ee_delta_pose --verbose --save_traj --max_retry 2 --reward_mode 'sparse' --record_rewards --num_envs 32 --count 100
# python -m mani_skill.trajectory.replay_trajectory --traj_path demos/PickCube-v1/motionplanning/trajectory.h5 --obs_mode rgb --target_control_mode pd_joint_delta_pos --verbose --save_traj --max_retry 2 --reward_mode 'sparse' --record_rewards --num_envs 32 --count 100
# python -m mani_skill.trajectory.replay_trajectory --traj_path demos/PushCube-v1/motionplanning/trajectory.h5 --obs_mode rgb --target_control_mode pd_ee_delta_pose --verbose --save_traj --max_retry 2 --reward_mode 'sparse' --record_rewards --num_envs 32 --count 500
# python -m mani_skill.trajectory.replay_trajectory --traj_path demos/StackCube-v1/rl/trajectory.none.pd_ee_delta_pose.physx_cuda.h5 --obs_mode rgb --target_control_mode pd_ee_delta_pose --verbose --save_traj --max_retry 2 --reward_mode 'sparse' --record_rewards --num_envs 32 --count 300
# python -m mani_skill.trajectory.replay_trajectory --traj_path demos/StackCube-v1/motionplanning/trajectory.h5 --obs_mode rgb --target_control_mode pd_joint_delta_pos --verbose --save_traj --max_retry 2 --reward_mode 'sparse' --record_rewards --num_envs 32 --count 100
# CUDA_VISIBLE_DEVICES=0 python train_rgbd.py --env-id StackCube-v1     --demo-path demos/StackCube-v1/motionplanning/trajectory.rgb.pd_ee_delta_pose.physx_cpu.h5     --control-mode "pd_ee_delta_pose" --sim-backend "physx_cpu" --num-demos 100 --max_episode_steps 200     --total_iters 100000 --obs-mode "rgb"     --exp-name diffusion_policy-StackCube-v1-rgb-100_motionplanning_demos-1     --demo_type=motionplanning --track

Writing video: ./test_save/0_0_0_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/1_1_0_150_False.mp4, Num steps: 150, Success: False


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Environment 0 done with reward 0.0027087628841400146, num dones: 16
Environment 1 done with reward 0.018450289964675903, num dones: 15
Writing video: ./test_save/2_0_1_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/3_1_1_150_False.mp4, Num steps: 150, Success: False
Environment 0 done with reward 0.0006744414567947388, num dones: 14
Environment 1 done with reward 0.007344335317611694, num dones: 13
Writing video: ./test_save/4_0_2_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/5_1_2_150_False.mp4, Num steps: 150, Success: False
Environment 0 done with reward 0.0005099326372146606, num dones: 12
Environment 1 done with reward 0.01807159185409546, num dones: 11
Writing video: ./test_save/6_0_3_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/7_1_3_150_False.mp4, Num steps: 150, Success: False


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Environment 0 done with reward 0.009738549590110779, num dones: 10
Environment 1 done with reward 0.002504289150238037, num dones: 9
Writing video: ./test_save/8_0_4_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/9_1_4_150_False.mp4, Num steps: 150, Success: False
Environment 0 done with reward 0.0008889734745025635, num dones: 8
Environment 1 done with reward 0.0012055039405822754, num dones: 7
Writing video: ./test_save/10_0_5_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/11_1_5_150_False.mp4, Num steps: 150, Success: False
Environment 0 done with reward 0.004766255617141724, num dones: 6
Environment 1 done with reward 6.490945816040039e-05, num dones: 5
Writing video: ./test_save/12_0_6_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/13_1_6_150_False.mp4, Num steps: 150, Success: False


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Environment 0 done with reward 0.003990381956100464, num dones: 4
Environment 1 done with reward 0.005703866481781006, num dones: 3
Writing video: ./test_save/14_0_7_150_False.mp4, Num steps: 150, Success: False
Writing video: ./test_save/15_1_7_150_False.mp4, Num steps: 150, Success: False


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (480, 529) to (480, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Environment 0 done with reward 0.049762338399887085, num dones: 2
Environment 1 done with reward 0.010378226637840271, num dones: 1


In [ ]:
import json

ds_json_path = "ManiSkill_Demonstrations/demos/PickCube-v1/teleop/trajectory.json"
with open(ds_json_path, "r") as f:
    env_info = json.load(f)
env = gym.make(env_info["env_id"], **env_info["env_kwargs"])
episode = env_info["episodes"][0] # picks the first
env.reset(**episode["reset_kwargs"])

In [6]:
from typing import Union
import h5py
import numpy as np
from torch.utils.data import Dataset
from tqdm import tqdm
import torch
from mani_skill.utils.io_utils import load_json
from mani_skill.utils import sapien_utils
from mani_skill.utils import common
from PIL import Image
import os

def load_h5_data(data):
    out = dict()
    for k in data.keys():
        if isinstance(data[k], h5py.Dataset):
            out[k] = data[k][:]
        else:
            out[k] = load_h5_data(data[k])
    return out

def to_tensors(x, device=None):
    """
    Converts numpy arrays or dicts of numpy arrays to torch tensors.
    If device is specified, moves the tensors to that device.
    """
    if isinstance(x, dict):
        return {k: to_tensors(v, device) for k, v in x.items()}
    elif isinstance(x, np.ndarray) and device is not None:
        tensor = torch.as_tensor(x).to(device)
        return tensor
    else:
        return x

class ManiSkillTrajectoryDataset(Dataset):
    """
    A general torch Dataset you can drop in and use immediately with just about any trajectory .h5 data generated from ManiSkill.
    This class simply is a simple starter code to load trajectory data easily, but does not do any data transformation or anything
    advanced. We recommend you to copy this code directly and modify it for more advanced use cases

    Args:
        dataset_file (str): path to the .h5 file containing the data you want to load
        load_count (int): the number of trajectories from the dataset to load into memory. If -1, will load all into memory
        success_only (bool): whether to skip trajectories that are not successful in the end. Default is false
        device: The location to save data to. If None will store as numpy (the default), otherwise will move data to that device
    """

    def __init__(self, dataset_file: str, load_count=-1, success_only: bool = False, device = None) -> None:
        self.dataset_file = dataset_file
        self.device = device
        self.data = h5py.File(dataset_file, "r")
        json_path = dataset_file.replace(".h5", ".json")
        self.json_data = load_json(json_path)
        self.episodes = self.json_data["episodes"]
        self.env_info = self.json_data["env_info"]
        self.env_id = self.env_info["env_id"]
        self.env_kwargs = self.env_info["env_kwargs"]
        if isinstance(load_count, int):
            self.load_dataset(load_count, success_only, device)
        else:
            pass    
        
    def load_dataset(self, load_count, success_only, device):
        self.obs = None
        self.actions = []
        self.terminated = []
        self.truncated = []
        self.success, self.fail, self.rewards = None, None, None
        if load_count == -1:
            load_count = len(self.episodes)
        for eps_id in tqdm(range(load_count)):
            eps = self.episodes[eps_id]
            if success_only: 
                assert "success" in eps, "episodes in this dataset do not have the success attribute, cannot load dataset with success_only=True"
                if not eps["success"]:
                    continue
            trajectory = self.data[f"traj_{eps['episode_id']}"]
            trajectory = load_h5_data(trajectory)
            eps_len = len(trajectory["actions"])
            
            # exclude the final observation as most learning workflows do not use it
            obs = common.index_dict_array(trajectory["obs"], slice(eps_len))
            if eps_id == 0:
                self.obs = obs
            else:
                self.obs = common.append_dict_array(self.obs, obs)

            self.actions.append(trajectory["actions"])
            self.terminated.append(trajectory["terminated"])
            self.truncated.append(trajectory["truncated"])

            # handle data that might optionally be in the trajectory
            if "rewards" in trajectory:
                if self.rewards is None:
                    self.rewards = [trajectory["rewards"]]
                else:
                    self.rewards.append(trajectory["rewards"])
            if "success" in trajectory:
                if self.success is None:
                    self.success = [trajectory["success"]]
                else:
                    self.success.append(trajectory["success"])
            if "fail" in trajectory:
                if self.fail is None:
                    self.fail = [trajectory["fail"]]
                else:
                    self.fail.append(trajectory["fail"])

        self.actions = np.vstack(self.actions)
        self.terminated = np.concatenate(self.terminated)
        self.truncated = np.concatenate(self.truncated)
        
        if self.rewards is not None:
            self.rewards = np.concatenate(self.rewards)
        if self.success is not None:
            self.success = np.concatenate(self.success)
        if self.fail is not None:
            self.fail = np.concatenate(self.fail)

        def remove_np_uint16(x: Union[np.ndarray, dict]):
            if isinstance(x, dict):
                for k in x.keys():
                    x[k] = remove_np_uint16(x[k])
                return x
            else:
                if x.dtype == np.uint16:
                    return x.astype(np.int32)
                return x
        
        # uint16 dtype is used to conserve disk space and memory
        # you can optimize this dataset code to keep it as uint16 and process that
        # dtype of data yourself. for simplicity we simply cast to a int32 so
        # it can automatically be converted to torch tensors without complaint
        self.obs = remove_np_uint16(self.obs)

        if device is not None:
            self.actions = to_tensors(self.actions, device=device)
            self.obs = to_tensors(self.obs, device=device)
            self.terminated = to_tensors(self.terminated, device=device)
            self.truncated = to_tensors(self.truncated, device=device)
            if self.rewards is not None:
                self.rewards = to_tensors(self.rewards, device=device)
            if self.success is not None:
                self.success = to_tensors(self.terminated, device=device)
            if self.fail is not None:
                self.fail = to_tensors(self.truncated, device=device)
                
                
    def __len__(self):
        return len(self.actions)


    def __getitem__(self, idx):
        action = to_tensors(self.actions[idx], device=self.device)
        obs = common.index_dict_array(self.obs, idx, inplace=False)

        res = dict(
            obs=obs,
            action=action,
            terminated=self.terminated[idx],
            truncated=self.truncated[idx],
        )
        if self.rewards is not None:
            res.update(reward=self.rewards[idx])
        if self.success is not None:
            res.update(success=self.success[idx])
        if self.fail is not None:
            res.update(fail=self.fail[idx])
        return res
            
            

/mnt/nfs3/caozhe/miniconda3/envs/verl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/nfs3/caozhe/miniconda3/envs/verl/lib/python3.10/site-packages/sapien/_vulkan_tricks.py:21: UserWarning: Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.
  warn("Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.")
/mnt/nfs3/caozhe/miniconda3/envs/verl/lib/python3.10/site-packages/sapien/_vulkan_tricks.py:37: UserWarning: Failed to find Vulkan ICD file. This is probably due to an incorrect or partial installation of the NVIDIA driver. SAPIEN will attempt to provide an ICD file anyway but it may not work.
  warn(


In [8]:
# small_dataset = ManiSkillTrajectoryDataset(dataset_file="ManiSkill_Demonstrations/demos/StackCube-v1/motionplanning/trajectory.rgb.pd_ee_delta_pose.physx_cpu.h5", load_count=10, success_only=False, device=None)

In [ ]:
# from internvl_eval.InternVL_eval_agent import video_writing, write_instruction_action, action_to_str

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# all_cameras = []
# ep_r = 0
# for data in small_dataset:
#     success = data['success']
#     reward = data['reward']
#     ep_r += reward
#     terminated = data['terminated']
#     truncated = data['truncated']
#     action = data['action']
#     camera = data['obs']['sensor_data']['base_camera']['rgb']
#     wrist_cam = data['obs']['sensor_data']['hand_camera']['rgb']
#     camera = np.concatenate([camera, wrist_cam], axis=1)  # concatenate the two cameras
#     instruction = f"R: {np.round(reward, 3)} EPR: {np.round(ep_r, 3)} Te: {terminated} Tr: {truncated} Su: {success}"
#     action = f"A: {action_to_str(action, 3)}"
#     camera = write_instruction_action(instruction, camera, action)
#     all_cameras.append(camera)
#     if terminated or truncated:
#         ep_r = 0
# video_writing([all_cameras], "./test_video.mp4", fps=10)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (960, 529) to (960, 544) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [ ]:
dataset = ManiSkillTrajectoryDataset(dataset_file="demos/StackCube-v1/motionplanning/trajectory.rgb.pd_ee_delta_pose.physx_cpu.h5", success_only=False, device=None)
generator = InternVLPretrainDatasetGenerator(
    dataset=dataset,
    save_path="stack_cubes"
    )
generator.cal_statistics()
generator.generation()